In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 28291796
paper_name = 'huseinovic_vos_2017' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [16]:
original_data1 = pd.read_excel('raw_data/pone.0173573.s003.xls', sheet_name='Integrated intensity', skiprows=5)

In [17]:
print('Original data dimensions: %d x %d' % (original_data1.shape))

Original data dimensions: 1536 x 25


In [18]:
original_data1.head()

,Spotnr,ORF,Integrated intensity,Integrated intensity.1,> MAX 95%?,> MAX 95%?.1,2x TRUE?,ORF.1,Integrated intensity.2,Integrated intensity.3,...,Integrated intensity.5,> MAX 95%?.4,> MAX 95%?.5,2x TRUE?.2,ORF.3,Integrated intensity.6,Integrated intensity.7,> MAX 95%?.6,> MAX 95%?.7,2x TRUE?.3
0,1,Empty,NaN,NaN,False,False,NaN,YLR149C,9.46964,74.6974,...,66.5875,False,False,NaN,YGL129C,131.685,210.2610,True,True,NaN
1,2,YLL040C,17.82400,54.1497,False,False,NaN,YKL063C,26.35900,26.2782,...,110.6840,False,True,NaN,YER175C,189.134,17.4955,True,False,NaN
2,3,YMR252C,7.60847,159.6140,False,True,NaN,YOR288C,10.05010,26.3817,...,114.6650,False,True,NaN,YOR128C,178.297,14.7712,True,False,NaN
3,4,YNL276C,7.80528,119.2080,False,True,NaN,YLR353W,4.90097,24.9784,...,137.6390,False,True,NaN,NaN,174.886,19.8658,True,False,NaN
4,5,YAL068C,9.30426,120.8870,False,True,NaN,YLR150W,4.45344,42.3538,...,130.1550,False,True,NaN,YGL131C,193.469,116.5830,True,True,NaN


In [20]:
original_data1a = original_data1[['ORF','Integrated intensity','Integrated intensity.1']]
original_data1b = original_data1[['ORF.1','Integrated intensity.2','Integrated intensity.3']]
original_data1c = original_data1[['ORF.2','Integrated intensity.4','Integrated intensity.5']]
original_data1d = original_data1[['ORF.3','Integrated intensity.6','Integrated intensity.7']]

cols = ['orf','data1','data2']
original_data1a.columns = cols
original_data1b.columns = cols
original_data1c.columns = cols
original_data1d.columns = cols

In [37]:
original_data1 = pd.concat([original_data1a, original_data1b, original_data1c, original_data1d], axis=0, ignore_index=True)

In [38]:
original_data1.head()

,orf,data1,data2
0,Empty,NaN,NaN
1,YLL040C,17.82400,54.1497
2,YMR252C,7.60847,159.6140
3,YNL276C,7.80528,119.2080
4,YAL068C,9.30426,120.8870


In [39]:
original_data1['orf'] = original_data1['orf'].astype(str)

In [40]:
# Eliminate all white spaces & capitalize
original_data1['orf'] = clean_orf(original_data1['orf'])

In [41]:
# Translate to ORFs 
original_data1['orf'] = translate_sc(original_data1['orf'], to='orf')

In [42]:
# Make sure everything translated ok
t = looks_like_orf(original_data1['orf'])
print(original_data1.loc[~t,])

               orf     data1     data2
index_input                           
0            EMPTY       NaN       NaN
5            EMPTY  32.11520  146.8810
18           EMPTY   9.56435   26.5134
23           EMPTY       NaN       NaN
50           EMPTY   4.32965  178.6270
...            ...       ...       ...
6139           NAN       NaN       NaN
6140           NAN       NaN       NaN
6141           NAN   5.18339       NaN
6142           NAN   2.74190       NaN
6143           NAN   5.79972   14.8270

[1366 rows x 3 columns]


In [43]:
original_data1 = original_data1.loc[t,]

In [44]:
for d in ['data1','data2']:
    original_data1[d] = pd.to_numeric(original_data1[d], errors='coerce')

In [45]:
original_data1['data'] = original_data1[['data1','data2']].mean(axis=1)

In [46]:
original_data1.set_index('orf', inplace=True)

In [47]:
original_data1 = original_data1[['data']].copy()

In [48]:
original_data1 = original_data1.groupby(original_data1.index).mean()

In [49]:
original_data1.shape

(4719, 1)

In [50]:
original_data1.head()

,data
orf,
YAL002W,18.400855
YAL004W,31.443100
YAL005C,18.005315
YAL007C,19.828900
YAL008W,21.557400


# Load & process the data (2)

In [55]:
original_data2 = pd.read_excel('raw_data/Krogan screen all data row sizes 27sep16.xlsx', sheet_name='30C day2 ', skiprows=3)
original_data3 = pd.read_excel('raw_data/Krogan screen all data row sizes 27sep16.xlsx', sheet_name='37C day2', skiprows=3)

In [58]:
print('Original data dimensions: %d x %d' % (original_data2.shape))
print('Original data dimensions: %d x %d' % (original_data3.shape))

Original data dimensions: 1536 x 21
Original data dimensions: 1536 x 24


In [59]:
original_data2.head()

,ORF,Name,Mutation,Plate 96,Row 96,Col 96,RowL 96,96 combined,96 combined L,Plate 384,...,Col 384,Row,Column,Raw colony size,Raw colony size.1,Raw colony size.2,Raw colony size.3,Raw colony size.4,Raw colony size.5,Raw colony size.6
0,YAL055W,PEX22,DELETION,1,1,1,A,1;1;1,1;A;1,1,...,1,1,1,828,927,885,934,935,843,397
1,YGR174C,CBP4,DELETION,5,1,1,A,5;1;1,5;A;1,2,...,1,1,2,566,507,493,481,509,472,223
2,YMR202W,ERG2,DELETION,2,1,1,A,2;1;1,2;A;1,1,...,2,1,3,571,619,631,561,645,514,238
3,YLR224W,YLR224W,DELETION,6,1,1,A,6;1;1,6;A;1,2,...,2,1,4,556,643,661,649,668,704,414
4,YAL053W,FLC2,DELETION,1,1,2,A,1;1;2,1;A;2,1,...,3,1,5,533,574,606,528,583,581,201


In [60]:
original_data2['orf'] = original_data2['ORF'].astype(str)
original_data3['orf'] = original_data3['ORF'].astype(str)

In [61]:
# Eliminate all white spaces & capitalize
original_data2['orf'] = clean_orf(original_data2['orf'])
original_data3['orf'] = clean_orf(original_data3['orf'])

In [62]:
# Translate to ORFs 
original_data2['orf'] = translate_sc(original_data2['orf'], to='orf')
original_data3['orf'] = translate_sc(original_data3['orf'], to='orf')

In [63]:
# Make sure everything translated ok
t = looks_like_orf(original_data2['orf'])
print(original_data2.loc[~t,])

                           ORF               Name Mutation  Plate 96  Row 96  \
index_input                                                                    
39           WT leu2D::KAN-GFP  WT leu2D::KAN-GFP       WT         6       1   
388              WT his3D::KAN      WT his3D::KAN       WT         1       3   
1504         WT leu2D::KAN-GFP  WT leu2D::KAN-GFP       WT        11       8   
1523             WT his3D::KAN      WT his3D::KAN       WT        16       8   

             Col 96 RowL 96 96 combined 96 combined L  Plate 384  ...  Row  \
index_input                                                       ...        
39               10       A      6;1;10        6;A;10          2  ...    1   
388               2       C       1;3;2         1;C;2          1  ...    9   
1504              5       A      11;8;5        11;H;5          3  ...   32   
1523              9       C      16;8;9        16;H;9          4  ...   32   

             Column  Raw colony size  Raw colony s

In [64]:
# Make sure everything translated ok
t = looks_like_orf(original_data3['orf'])
print(original_data3.loc[~t,])

                           ORF               Name Mutation  Plate 96  Row 96  \
index_input                                                                    
39           WT leu2D::KAN-GFP  WT leu2D::KAN-GFP       WT         6       1   
388              WT his3D::KAN      WT his3D::KAN       WT         1       3   
1504         WT leu2D::KAN-GFP  WT leu2D::KAN-GFP       WT        11       8   
1523             WT his3D::KAN      WT his3D::KAN       WT        16       8   

             Col 96 RowL 96 96 combined 96 combined L  Plate 384  ...  \
index_input                                                       ...   
39               10       A      6;1;10        6;A;10          2  ...   
388               2       C       1;3;2         1;C;2          1  ...   
1504              5       A      11;8;5        11;H;5          3  ...   
1523              9       C      16;8;9        16;H;9          4  ...   

             Raw colony size.1  Raw colony size.2  Raw colony size.3  \
index_in

In [65]:
original_data2 = original_data2.loc[t,]
original_data3 = original_data3.loc[t,]

In [68]:
original_data3.head()

,ORF,Name,Mutation,Plate 96,Row 96,Col 96,RowL 96,96 combined,96 combined L,Plate 384,...,Raw colony size.1,Raw colony size.2,Raw colony size.3,Raw colony size.4,Raw colony size.5,Raw colony size.6,Raw colony size.7,Raw colony size.8,Raw colony size.9,orf
index_input,,,,,,,,,,,,,,,,,,,,,
0,YAL055W,PEX22,DELETION,1,1,1,A,1;1;1,1;A;1,1,...,657,557,552,594,554,499,399,226,112,YAL055W
1,YGR174C,CBP4,DELETION,5,1,1,A,5;1;1,5;A;1,2,...,588,423,392,410,431,342,308,190,133,YGR174C
2,YMR202W,ERG2,DELETION,2,1,1,A,2;1;1,2;A;1,1,...,538,458,436,403,439,263,238,139,87,YMR202W
3,YLR224W,YLR224W,DELETION,6,1,1,A,6;1;1,6;A;1,2,...,568,527,507,557,562,521,460,322,250,YLR224W
4,YAL053W,FLC2,DELETION,1,1,2,A,1;1;2,1;A;2,1,...,494,429,425,470,419,336,266,144,117,YAL053W


In [67]:
for d in ['Raw colony size','Raw colony size.1','Raw colony size.2','Raw colony size.3','Raw colony size.4','Raw colony size.5','Raw colony size.6']:
    original_data2[d] = pd.to_numeric(original_data2[d], errors='coerce')

In [69]:
for d in ['Raw colony size','Raw colony size.1','Raw colony size.2','Raw colony size.3','Raw colony size.4','Raw colony size.5','Raw colony size.6','Raw colony size.7','Raw colony size.8','Raw colony size.9']:
    original_data3[d] = pd.to_numeric(original_data3[d], errors='coerce')

In [70]:
original_data3['0'] = original_data3[['Raw colony size','Raw colony size.1']].mean(axis=1)
original_data3['50'] = original_data3[['Raw colony size.2','Raw colony size.3']].mean(axis=1)
original_data3['60'] = original_data3[['Raw colony size.4','Raw colony size.5']].mean(axis=1)
original_data3['70'] = original_data3[['Raw colony size.6','Raw colony size.7']].mean(axis=1)
original_data3['80'] = original_data3[['Raw colony size.8','Raw colony size.9']].mean(axis=1)

In [71]:
original_data2.set_index('orf', inplace=True)
original_data3.set_index('orf', inplace=True)

In [74]:
original_data2 = original_data2[['Raw colony size','Raw colony size.1','Raw colony size.2','Raw colony size.3','Raw colony size.4','Raw colony size.5','Raw colony size.6']].copy()
original_data2.columns = ['0','50','60','70','80','90','100']

In [75]:
original_data3 = original_data3[['0','50','60','70','80']].copy()

In [76]:
original_data4 = original_data2.join(original_data3, how='outer', lsuffix='_30', rsuffix='_37')

In [78]:
original_data4.shape

(1550, 12)

In [79]:
original_data4 = original_data4.groupby(original_data4.index).mean()

In [80]:
original_data4.shape

(1523, 12)

In [81]:
original_data4.head()

,0_30,50_30,60_30,70_30,80_30,90,100,0_37,50_37,60_37,70_37,80_37
orf,,,,,,,,,,,,
YAL002W,290.0,313.0,318.0,354.0,350.0,362.0,143.0,300.0,250.0,248.5,167.0,92.5
YAL007C,273.0,326.0,372.0,343.0,344.0,369.0,205.0,311.5,277.0,297.0,239.5,132.0
YAL010C,488.0,510.0,523.0,499.0,502.0,476.0,219.0,495.5,411.0,359.0,244.5,99.0
YAL011W,508.0,369.0,197.0,69.0,144.0,78.0,20.0,487.5,197.0,36.0,56.0,34.5
YAL013W,230.0,243.0,256.0,251.0,258.0,251.0,125.0,238.0,192.0,196.5,111.5,46.5


In [82]:
# Normalize to dose 0
for c in ['50_30','60_30','70_30','80_30','90','100']:
    original_data4[c] = original_data4[c] / original_data4['0_30']
    
for c in ['50_37','60_37','70_37','80_37']:
    original_data4[c] = original_data4[c] / original_data4['0_37']

In [84]:
original_data4.drop(columns=['0_30','0_37'], inplace=True)

In [85]:
original_data4.head()

,50_30,60_30,70_30,80_30,90,100,50_37,60_37,70_37,80_37
orf,,,,,,,,,,
YAL002W,1.079310,1.096552,1.220690,1.206897,1.248276,0.493103,0.833333,0.828333,0.556667,0.308333
YAL007C,1.194139,1.362637,1.256410,1.260073,1.351648,0.750916,0.889246,0.953451,0.768860,0.423756
YAL010C,1.045082,1.071721,1.022541,1.028689,0.975410,0.448770,0.829465,0.724521,0.493441,0.199798
YAL011W,0.726378,0.387795,0.135827,0.283465,0.153543,0.039370,0.404103,0.073846,0.114872,0.070769
YAL013W,1.056522,1.113043,1.091304,1.121739,1.091304,0.543478,0.806723,0.825630,0.468487,0.195378


In [86]:
# Merge all datasets
original_data = original_data1.join(original_data4, how='outer')

In [88]:
original_data.shape

(4796, 11)

In [90]:
original_data.head()

,data,50_30,60_30,70_30,80_30,90,100,50_37,60_37,70_37,80_37
orf,,,,,,,,,,,
YAL002W,18.400855,1.079310,1.096552,1.22069,1.206897,1.248276,0.493103,0.833333,0.828333,0.556667,0.308333
YAL004W,31.443100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YAL005C,18.005315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YAL007C,19.828900,1.194139,1.362637,1.25641,1.260073,1.351648,0.750916,0.889246,0.953451,0.768860,0.423756
YAL008W,21.557400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Prepare the final dataset

In [89]:
data = original_data.copy()

In [91]:
dataset_ids = [22075, 22088, 22089, 22090, 22091, 22092, 22087, 22076, 22082, 22083, 22084]
datasets = datasets.reindex(index=dataset_ids)

In [92]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [93]:
data.head()

dataset_id,22075,22088,22089,22090,22091,22092,22087,22076,22082,22083,22084
data_type,value,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,
YAL002W,18.400855,1.079310,1.096552,1.22069,1.206897,1.248276,0.493103,0.833333,0.828333,0.556667,0.308333
YAL004W,31.443100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YAL005C,18.005315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YAL007C,19.828900,1.194139,1.362637,1.25641,1.260073,1.351648,0.750916,0.889246,0.953451,0.768860,0.423756
YAL008W,21.557400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Subset to the genes currently in SGD

In [94]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 3


In [95]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22075,22088,22089,22090,22091,22092,22087,22076,22082,22083,22084
,data_type,value,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,
2,YAL002W,18.400855,1.079310,1.096552,1.22069,1.206897,1.248276,0.493103,0.833333,0.828333,0.556667,0.308333
1863,YAL004W,31.443100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,YAL005C,18.005315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,YAL007C,19.828900,1.194139,1.362637,1.25641,1.260073,1.351648,0.750916,0.889246,0.953451,0.768860,0.423756
6,YAL008W,21.557400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Normalize

In [96]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [97]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [98]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,22075,22088,22089,22090,22091,22092,22087,22076,22082,22083,...,22088,22089,22090,22091,22092,22087,22076,22082,22083,22084
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
2,YAL002W,18.400855,1.079310,1.096552,1.22069,1.206897,1.248276,0.493103,0.833333,0.828333,0.556667,...,-0.453765,-0.475255,0.188387,-0.113196,-0.033283,-0.848890,-0.166014,-0.172676,-0.570759,0.019759
1863,YAL004W,31.443100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,YAL005C,18.005315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,YAL007C,19.828900,1.194139,1.362637,1.25641,1.260073,1.351648,0.750916,0.889246,0.953451,0.768860,...,0.349211,0.868551,0.332140,0.109546,0.310228,0.261912,0.171986,0.322076,0.087190,0.449815
6,YAL008W,21.557400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Print out

In [99]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [100]:
from IO.save_data_to_db3 import *

In [101]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/11 [00:00<?, ?it/s]

Deleting all datasets for PMID 28291796...
Inserting the new data...


100%|██████████| 11/11 [01:01<00:00,  5.57s/it]

Updating the data_modified_on field...
